In [1]:
import enum
import cv2
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [2]:
import traitlets
import ipywidgets.widgets as widgets
import time
import threading
import inspect
import ctypes

origin_widget = widgets.Image(format='jpeg', width=320, height=240)
mask_widget = widgets.Image(format='jpeg',width=320, height=240)
result_widget = widgets.Image(format='jpeg',width=320, height=240)
image_container = widgets.HBox([origin_widget, mask_widget, result_widget])

display(image_container)

In [3]:
def get_color(img):
    H = []
    color_name={}
    img = cv2.resize(img, (640, 480), )

    HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    cv2.rectangle(img, (280, 180), (360, 260), (0, 255, 0), 2)

    for i in range(280, 360):
        for j in range(180, 260): H.append(HSV[j, i][0])

    H_min = min(H);H_max = max(H)

    if H_min >= 0 and H_max <= 10 or H_min >= 156 and H_max <= 180:
        color_name['name'] = 'red'

    elif H_min >= 26 and H_max <= 34:
        color_name['name'] = 'yellow'

    elif H_min >= 35 and H_max <= 78:
        color_name['name'] = 'green'

    elif H_min >= 100 and H_max <= 124:
        color_name['name'] = 'blue'

    return img, color_name

In [ ]:
import cv2
import numpy as np
import ipywidgets.widgets as widgets

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(5, 30)

cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))

color_lower = np.array([0, 43, 46])
color_upper = np.array([10, 255, 255])

def Color_Recongnize():

    while 1:
        ret, frame = cap.read()
        frame, color_name = get_color(frame)

        if len(color_name)==1:
            global color_lower
            global color_upper

            if color_name['name'] == 'yellow':
                color_lower = np.array([26, 43, 46])
                color_upper = np.array([34, 255, 255])
         
            elif color_name['name'] == 'red':
                color_lower = np.array([0, 43, 46])
                color_upper = np.array([10, 255, 255])
             
            elif  color_name['name'] == 'green':
                color_lower = np.array([35, 43, 46])
                color_upper = np.array([77, 255, 255])
             
            elif color_name['name'] == 'blue':
                color_lower=np.array([100, 43, 46])
                color_upper = np.array([124, 255, 255])

        origin_widget.value = bgr8_to_jpeg(frame) 
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, color_lower, color_upper)
        mask_widget.value = bgr8_to_jpeg(mask)
        res = cv2.bitwise_and(frame, frame, mask=mask)
        result_widget.value = bgr8_to_jpeg(res)

        time.sleep(0.01)

    cap.release()